<a href="https://colab.research.google.com/github/AnetaGancheva/Notebooks/blob/main/MultiAgentSystem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculate the distance to the furthest Irish pub in the world

A Multi Agent System practice:

Current scope: System of agents will calculate the travel distance between current position and the furthest Irish pub in the world.

In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import math
from typing import Tuple
from smolagents import tool

@tool
def calculate_distance(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
) -> float:

    """
    A function to calculate the distance between two points on Earth based on their coordinates.

    Args:

    origin_coords: Tuple of (latitude, longitude) for starting point
    destination_coords: Tuple of (latitude, longitude) for end point

    Returns:
    float: the calculated distance in km between the two points

    Example:
        >>> # Varna, Bulgaria (43.2141° N, 27.9147° E) to Santiago, Chille (33.4489° S, 70.6693° W)
        >>> result = calculate_distance((43.2141, 27.9147), (-33.4489, -70.6693))
    """

    # Convert degrees to radians function
    def to_rads(degrees: float) -> float:
      return (degrees*(math.pi/180.0))

    # Extract coordinates
    lat_orig, lon_orig = map(to_rads, origin_coords)
    lat_dest, lon_dest = map(to_rads, destination_coords)

    # Earth's radius (km)
    EARTH_RADIUS_KM = 6371.0

    # Apply Haversine formula to get distance
    dlat = lat_dest - lat_orig
    dlon = lon_dest - lon_orig

    a = (
        math.sin(dlat/2) ** 2 + math.cos(lat_orig) * math.cos(lat_dest) * math.sin(dlon/2) ** 2
    )

    c = 2 * math.asin(math.sqrt(a))

    distance = EARTH_RADIUS_KM * c

    return distance


In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, VisitWebpageTool

model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [5]:
web_agent = CodeAgent(
    model= model,
    tools=[ DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_distance ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [6]:
import os
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel
from PIL import Image


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

In [12]:
manager_agent = CodeAgent(
    model=HfApiModel("https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/"),
    tools=[calculate_distance],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15
)

In [13]:
manager_agent.run("""
Find the locations of all Irish pubs in the world and calculate the distance to Varna, Bulgaria (43.2141° N, 27.9147° E).
Represent this as a spatial map of the world, with the locations represented as scatter ploints with a color that depends on travel time.

Here is an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you will see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the locations of all Irish pubs in the world and calculate the distance to Varna, Bulgaria (43.2141° N,    │
│ 27.9147° E).                                                                                                    │
│ Represent this as a spatial map of the world, with the locations represented as scatter ploints with a color    │
│ that depends on travel time.                                                                                    │
│                                                                                                                 │
│ Here is an example of how to plot and return a map:                                                             │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you will see it.     │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud/ ──────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Generate a task for the web_agent to find a comprehensive list of all Irish pubs in the world, including their 
names and locations (latitude and longitude).
2. Retrieve the list of Irish pubs from the web_agent.
3. For each pub in the list, use the calculate_distance tool to calculate the distance from the pub's location to 
Varna, Bulgaria (43.2141° N, 27.9147° E).
4. Organize the data into a structured format, including pub names, their coordinates, distances to Varna, 
Bulgaria, and estimated travel times.
5. Use Plotly Express to create a spatial map of the world, with the locations of the pubs represented as scatter 
points.
6. Color the scatter points based on the travel times, using a color scale that maps travel times to colors.
7. Customize the plot to include text labels for each pub, set an appropriate size for the scatter points, and 
select an appropriate zoom level.
8. Display the map using the `fig.show()` method.
9. Save the final map as an image file using `fig.write_image("saved_image.png")`.
10. Provide the final answer using the final_answer function, passing the plot as the answer.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought:                                                                                                           
To begin, I will generate a task for the web_agent to find a comprehensive list of all Irish pubs in the world,    
including their names and locations (latitude and longitude). Since gathering a global list of Irish pubs is a     
complex and extensive task, I'll request the web_agent to gather as many(pub details as possible, from reliable    
sources like pub directories, tourism websites, or business locations databases.                                   
                                                                                                                   
I'll also request the web_agent to include as many specific fields as possible in the data, such as:               
- Pub名称 (Name of the Pub)                                                                                        
- Latitude (Lat)                                                                                                   
- Longitude (Lon)                                                                                                  
                                                                                                                   
The quality and comprehensiveness of the data returned by the web_agent will significantly impact the success of   
the subsequent steps.                                                                                              
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
task_for_web_agent = """                                                                                           
Please gather a comprehensive list of all Irish pubs in the world, including their names and locations (latitude   
and longitude).                                                                                                    
The data should be formatted as CSV or JSON, with each row representing a pub and including fields for:            
- Pub Name                                                                                                         
- Latitude                                                                                                         
- Longitude                                                                                                        
Return the data as a string in CSV format [38;2;230;237;243;48;2;13;17;2

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task_for_web_agent = """                                                                                         
  Please gather a comprehensive list of all Irish pubs in the world, including their names and locations           
  (latitude and longitude).                                                                                        
  The data should be formatted as CSV or JSON, with each row representing a pub and including fields for:          
  - Pub Name                                                                                                       
  - Latitude                                                                                                       
  - Longitude                                                                                                      
  Return the data as a string in CSV format (with headers), and ensure that the data is accurate and               
  comprehensive to the best of your ability.                                                                       
  """                                                                                                              
  web_agent(task=task_for_web_agent)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Code execution failed at line 'import matplotlib.pyplot as plt' due to: InterpreterError: Import of 
matplotlib.pyplot is not allowed. Authorized imports are: ['datetime', 'unicodedata', 'random', 'queue', 
'geopandas', 'collections', 'json', 'stat', 'numpy', 'pandas', 'shapely', 'time', 'plotly', 'itertools', 'math', 
'statistics', 're'\]

[Step 9: Duration 42.96 seconds| Input tokens: 71,347 | Output tokens: 4,877]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 10: Duration 4.49 seconds| Input tokens: 86,328 | Output tokens: 6,289]

KeyboardInterrupt: 